<a href="https://colab.research.google.com/github/hyeryn/Natural-Language/blob/master/crawling_movie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install selenium

     |████████████████████████████████| 911kB 15.4MB/s 


In [3]:
from bs4 import BeautifulSoup 
import requests 
from selenium import webdriver as wd 
from selenium.webdriver.common.keys import Keys 
import time 
import re 
import pandas as pd

In [5]:
#원하는 페이지까지 링크들을 받아오는 함수
def getPageLinks(pageRange): 
  links = [] 
  
  for pageNo in range(pageRange): 
    url = "https://series.naver.com/movie/recentList.nhn?orderType=sale&sortingType=&tagCode=&page=" + str(pageNo + 1) 
    req = requests.get(url) 
    soup = BeautifulSoup(req.text, 'lxml') 
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') 
    
    for movielink in movielinks: 
      link = str(movielink.get('href')) 
      links.append("https://series.naver.com"+link) 
  return links

In [16]:
print(getPageLinks(3))

['https://series.naver.com/movie/detail.nhn?productNo=2909332', 'https://series.naver.com/movie/detail.nhn?productNo=2747742', 'https://series.naver.com/movie/detail.nhn?productNo=3293948', 'https://series.naver.com/movie/detail.nhn?productNo=2686058', 'https://series.naver.com/movie/detail.nhn?productNo=1405968', 'https://series.naver.com/movie/detail.nhn?productNo=2273058', 'https://series.naver.com/movie/detail.nhn?productNo=1781025', 'https://series.naver.com/movie/detail.nhn?productNo=4034051', 'https://series.naver.com/movie/detail.nhn?productNo=1373317', 'https://series.naver.com/movie/detail.nhn?productNo=1775085', 'https://series.naver.com/movie/detail.nhn?productNo=2996569', 'https://series.naver.com/movie/detail.nhn?productNo=4113612', 'https://series.naver.com/movie/detail.nhn?productNo=2743970', 'https://series.naver.com/movie/detail.nhn?productNo=1783636', 'https://series.naver.com/movie/detail.nhn?productNo=4465876', 'https://series.naver.com/movie/detail.nhn?productNo=1

In [6]:
#원하는 시작페이지부터 끝나는 페이지 사이의 링크 받아오는 함수(태그 속성값 활용)
def getPageLinksWantRange(startPageNo, lastPageNo): 
  links = [] 
  return_links = [] 

  for pageNo in range(startPageNo-1, lastPageNo): 
    url = "https://series.naver.com/movie/recentList.nhn?orderType=sale&sortingType=&tagCode=&page=" + str(pageNo + 1) 
    req = requests.get(url) 
    soup = BeautifulSoup(req.text, 'lxml')
    movielinks = soup.select('div.lst_thum_wrap ul li a[href]') 
    
    for movielink in movielinks: 
      link = str(movielink.get('href')) 
      links.append("https://series.naver.com"+link) 
      
  return links


In [17]:
print(getPageLinksWantRange(1,2))

['https://series.naver.com/movie/detail.nhn?productNo=2909332', 'https://series.naver.com/movie/detail.nhn?productNo=2747742', 'https://series.naver.com/movie/detail.nhn?productNo=3293948', 'https://series.naver.com/movie/detail.nhn?productNo=2686058', 'https://series.naver.com/movie/detail.nhn?productNo=1405968', 'https://series.naver.com/movie/detail.nhn?productNo=2273058', 'https://series.naver.com/movie/detail.nhn?productNo=1781025', 'https://series.naver.com/movie/detail.nhn?productNo=4034051', 'https://series.naver.com/movie/detail.nhn?productNo=1373317', 'https://series.naver.com/movie/detail.nhn?productNo=1775085', 'https://series.naver.com/movie/detail.nhn?productNo=2996569', 'https://series.naver.com/movie/detail.nhn?productNo=4113612', 'https://series.naver.com/movie/detail.nhn?productNo=2743970', 'https://series.naver.com/movie/detail.nhn?productNo=1783636', 'https://series.naver.com/movie/detail.nhn?productNo=4465876', 'https://series.naver.com/movie/detail.nhn?productNo=1

In [31]:
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:6 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:13 https://developer.download.n

In [34]:
# 받아온 데이터를 바탕으로 dataframe 만들고 중복값 제거 후 return
def getMovieDataFromNaverSeries(links): 
  title_infos = [] 
  content_infos = [] 
  genre_infos = []
  score_infos = [] 
  date_infos = [] 

  url2 = "https://www.naver.com" 

  from selenium import webdriver
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
  #driver = wd.Chrome(executable_path=r"/content/chromedriver.exe") 
  driver.get(url2) 
  time.sleep(3.0) # 30 

  driver.find_element_by_css_selector('body').send_keys(Keys.CONTROL + "t") 

  for link in links: 
    driver.switch_to.window(driver.window_handles[-1]) 
    time.sleep(0.1)
    driver.get(link) 
    time.sleep(0.1) 
    driver.switch_to.window(driver.window_handles[0])
    time.sleep(0.3) 

    html_source = driver.page_source 
    html_soup = BeautifulSoup(html_source, 'lxml') 

    flag = html_soup.text[0:10] 

    newflag = "".join(flag) 
    newflag = newflag.replace('\n', '') 

    if newflag == '네이버': 
      time.sleep(1.0) 

      score = driver.find_element_by_css_selector('div.score_area > em ') 
      score = float(score.text) 
      score = int(score) 
      score_infos.append(score) 

      genre = driver.find_element_by_css_selector('li.info_lst > ul > li:nth-child(4)').get_attribute('textContent') 
      genre = genre.replace('장르','') 
      genre = genre.split('/') 
      genre_infos.append(genre) 

      text = driver.find_element_by_css_selector('span.al_r > a') #.click() 
      # review_url = text.get_attribute('href') 
      # review_url = review_url.replace('basic', 'pointWriteFormList') 
      # review_url = review_url + '&type=after&onlyActualPointYn=N&order=newest&page=1'

      movieInfoUrl = text.get_attribute('href')
      movie_req = requests.get(movieInfoUrl) 			
      movie_soup = BeautifulSoup(movie_req.text, 'lxml') 
      titles = movie_soup.select('div.mv_info > h3.h_movie > a') 
      temp_titles = [] 

      for title in titles:
        temp = title.text 
        temp = temp.replace('상영중', '') 
        temp = temp.replace('\n', '') 				
        temp_titles.append(temp) 

      if '' in temp_titles or ' ' in temp_titles: 
        temp_titles.remove('') 

      temp_titles = set(temp_titles) 
      temp_titles = list(temp_titles) 
      temp_titles = [x for x in temp_titles if x is not ''] 			
      title_infos.append(list(temp_titles)[0]) 

      contents_texts = movie_soup.select('div.story_area > p.con_tx') 
      if len(contents_texts) == 0: 
        content_infos.append("줄거리 오류") 			
      else: 
        for contents in contents_texts: 
          temp = contents.text 
          temp = temp.replace('\r', '') 					
          temp = temp.replace('\xa0', '') 
          content_infos.append(temp) 		
    elif newflag == '네이버 :': 
      adult_movies.append(link) 
  print(len(score_infos), len(genre_infos), len(content_infos)) 
  driver.close() 
  movie_dic = {"평점":score_infos,"장르":genre_infos, "줄거리":content_infos} 
  movie_df = pd.DataFrame(movie_dic, index=title_infos) 
  movie_df2 = movie_df.drop_duplicates("줄거리", keep='first') 
 
  return movie_df2

In [35]:
print(getMovieDataFromNaverSeries(getPageLinks(3)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: DeprecationWarning: use options instead of chrome_options
  app.launch_new_instance()


NoSuchElementException: ignored

In [14]:
def dftoCsv(movie_df, num):
  try:
    movie_df.to_csv(('mmovie_data'+str(num)+'.csv'), sep=',',na_rep='NaN', encoding='euc-kr')
  except:
    print("Error")